In [2]:
import pandas as pd
import copy
import numpy as np
from collections import Counter

# TODOs

## Partner

- <span style="color:red">BUG: more wins than games for (probably) games with 3rd player</span>
- <span style="color:orange">IMPORTANT: also NEMESIS -> already implemented -> see R</span>
- <span style="color:orange">IMPORTANT: handle draws: currently each game and win is counted -> draws count as loss against opponent and negatively impact overall winrate</span>
- specify when 3 ppl played together
-> re-structure probably: build df iteratively 

## Misc

In [3]:
spiele = pd.read_excel('Spiele.xlsx')
plätze = pd.read_excel('Platzierungen.xlsx')

spiele.fillna(value="", inplace = True)
plätze.fillna(value="", inplace = True)

# rename columns
spiele = spiele.rename(columns = {'Player 1': 'p1', 'Player 2': 'p2', 'Player 3': 'p3', 'Player 4': 'p4', 
                                  'Player 5': 'p5', 'Player 6': 'p6', 'Teamname 1': 'tname1', 'Teamname 2': 'tname2',
                                 'Datum': 'datum', 'W/L Team 1': 'WLt1', 'W/L Team 2': 'WLt2', 'Becher übrig': 'becher',
                                 'Trickshot': 'trickshot', 'Fenster Team': 'fenster', 'Verlängerung': 'verl',
                                 'Becher P1': 'p1Becher', 'Becher P2': 'p2Becher', 'Becher P3': 'p3Becher', 
                                 'Becher T1': 't1Becher', 'Becher P4': 'p4Becher', 'Becher P5': 'p5Becher',
                                  'Becher P6': 'p6Becher', 'Becher T2': 't2Becher', '1v1': '1v1', 
                                  'X_Runden Regel': '5called', 'Eigentore': 'Eigentore'})

# Apply str.strip() to remove trailing spaces from each cell
spiele = spiele.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# define player columns
player_cols  = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

# define all player(names)
players = pd.unique(spiele[player_cols].values.flatten())
players = players.tolist()
# remove empty name
players = [value for value in players if value]
# sort alphabetically
players.sort()

In [4]:
# make sure either both teams lost (X Runden gecalled) or exactly one team lost and excatly one team won
rule = spiele.apply(lambda row: (row['WLt1'], row['WLt2']) in [(0, 1), (1, 0), (0, 0)], axis=1)

if rule.all():
    print("Fine")
else:
    print('Fehler')


Fine


In [60]:
players

['Aaron',
 'Alina',
 'André',
 'Basti',
 'Chris',
 'Cuba',
 'Daisy',
 'Eggi',
 'Emilio',
 'Friedrich',
 'Gina',
 'Hannes',
 'Henny',
 'Jonte',
 'Konni',
 'Konrad',
 'Leon',
 'Lin',
 'Lorenz',
 'Luca',
 'Marian',
 'Marie',
 'Max',
 'Merlin',
 'Michi',
 'Moritz',
 'Mümmel',
 'Nikita',
 'Nora',
 'Ole',
 'Olek',
 'Phia',
 'Philipp',
 'Richard',
 'Tabea',
 'Timo',
 'Tobi',
 'Valdemar',
 'Verena',
 'Vincent',
 'Zoë']

In [61]:
spiele

,p1,p2,p3,p4,p5,p6,tname1,tname2,datum,WLt1,...,p2Becher,p3Becher,t1Becher,p4Becher,p5Becher,p6Becher,t2Becher,1v1,X-Runden Regel,Eigentore
0,Basti,Valdemar,,Emilio,Philipp,,Einmal mit und einmal ohne Socken,Saufomat,2021-11-26,0,...,,,,,,,,,,
1,Phia,Friedrich,,André,Timo,,Team Pferd *hüüh*,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
2,Phia,Friedrich,,Basti,Valdemar,,Team Pferd *hüüh*,Einmal mit und einmal ohne Socken,2021-11-26,1,...,,,,,,,,,,
3,Emilio,Philipp,,André,Timo,,Saufomat,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
4,Emilio,Philipp,,Phia,Friedrich,,Saufomat,Team Pferd *hüüh*,2021-11-26,0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Phia,André,,Philipp,Valdemar,,Pieps ÖHH & KraKraKra,Borosboiiis,2024-05-26,1,...,3.0,,6.0,2.0,2.0,,4.0,,,
228,Phia,Chris,,André,Marian,,DD-Detlef und Doris,Geckabor Gangster,2024-05-26,0,...,2.0,,3.0,3.0,3.0,,6.0,,,
229,Emilio,Valdemar,,Phia,Chris,,FDP Ferbrecher,DD-Detlef und Doris,2024-05-26,1,...,1.0,,6.0,2.0,0.0,,2.0,,,
230,André,Marian,,Emilio,Valdemar,,Geckabor Gangster,FDP Ferbrecher,2024-05-26,0,...,4.0,,6.0,4.0,5.0,,9.0,,,


In [62]:
plätze

,Player 1,Player 2,Player 3,Teamname,Datum,Platzierung,Anzahl Teams,Draft,Datum unsicher
0,Jonte,Valdemar,,Schöne & Biest,2021-08-07,1,4,0,
1,Michi,André,,Valdemars Albtraum,2021-08-07,2,4,0,
2,Daisy,Marie,,Immo-rich,2021-08-07,4,4,0,
3,Phia,Emilio,,Support Main,2021-08-07,2,4,0,
4,Basti,Valdemar,,Einmal mit und einmal ohne Socken,2021-11-26,3,4,1,
...,...,...,...,...,...,...,...,...,...
148,Phia,André,,Pieps ÖHH & KraKraKra,2024-05-26,1,4,1,
149,Marian,Chris,,TT-Torsten und Thorben,2024-05-26,3,4,1,
150,Phia,Chris,,DD-Detlef und Doris,2024-05-26,3,3,1,
151,Emilio,Valdemar,,FDP Ferbrecher,2024-05-26,1,3,1,


In [130]:
def winrate(player):
    df = spiele.copy(deep=True)
    pattern = rf'\b{player}\b'
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    
    team = -1
    games = 0
    wins = 0
    partner = {elem: [0, 0] for elem in players}
    # opponents: 3rd value = winrate AGAINST opponent
    opponents = {elem: [0, 0] for elem in players}
    player_number = 'p0'
    
    for i in range(len(df)):
        partner1, partner2 = '', ''
        
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            partner1, partner2 = df.loc[i, 'p2'], df.loc[i, 'p3']
            opponent1, opponent2, opponent3, = df.loc[i, 'p4'], df.loc[i, 'p5'], df.loc[i, 'p6']
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            partner1, partner2 = df.loc[i, 'p1'], df.loc[i, 'p3']
            opponent1, opponent2, opponent3, = df.loc[i, 'p4'], df.loc[i, 'p5'], df.loc[i, 'p6']
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            partner1, partner2 = df.loc[i, 'p1'], df.loc[i, 'p2']
            opponent1, opponent2, opponent3, = df.loc[i, 'p4'], df.loc[i, 'p5'], df.loc[i, 'p6']
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            partner1, partner2 = df.loc[i, 'p5'], df.loc[i, 'p6']
            opponent1, opponent2, opponent3, = df.loc[i, 'p1'], df.loc[i, 'p2'], df.loc[i, 'p3']
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            partner1, partner2 = df.loc[i, 'p4'], df.loc[i, 'p6']
            opponent1, opponent2, opponent3, = df.loc[i, 'p1'], df.loc[i, 'p2'], df.loc[i, 'p3']
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            partner1, partner2 = df.loc[i, 'p4'], df.loc[i, 'p5']
            opponent1, opponent2, opponent3, = df.loc[i, 'p1'], df.loc[i, 'p2'], df.loc[i, 'p3']
        else:
            print(f"Player {player} not found in row {i}")
            continue
        
        if partner1:
            partner[partner1][0] += 1
        if partner2:
            partner[partner2][0] += 1
        # add 3 player teams (don't know how yet)
        #if partner1 & partner2:
        #    partner[partner1 + "&"+ partner2][0] = 1
        
        if opponent1:
            opponents[opponent1][0] += 1
        if opponent2:
            opponents[opponent2][0] += 1
        if opponent3:
            opponents[opponent3][0] += 1
        
        games += 1
        if team == 1:
            current_win = df.loc[i, 'WLt1']
        else:
            current_win = df.loc[i, 'WLt2']
        
        if current_win:
            wins += 1
            if partner1:
                partner[partner1][1] += 1
            if partner2:
                partner[partner2][1] += 1
            if opponent1:
                opponents[opponent1][1] += 1
            if opponent2:
                opponents[opponent2][1] += 1
            if opponent3:
                opponents[opponent3][1] += 1
            
    
    winrate = wins / games * 100
    print(f'Total games: {games}')
    print(f'Winrate {player}: {winrate:.2f}%\n')
    
    for elem in partner:
        games_tmp = partner[elem][0]
        wins_tmp = partner[elem][1]
        if games_tmp != 0:
            winrate_tmp = wins_tmp / games_tmp
        else:
            winrate_tmp = 0
        partner[elem].append(winrate_tmp)
        
    for elem in opponents:
        games_tmp = opponents[elem][0]
        wins_tmp = opponents[elem][1]
        if games_tmp != 0:
            winrate_tmp = wins_tmp / games_tmp
        else:
            winrate_tmp = 0
        opponents[elem].append(winrate_tmp)
    
    return partner, opponents

In [133]:
phia_partner, _ = winrate('Phia')

Total games: 120
Winrate Phia: 53.33%



In [134]:
phia_partner

{'Aaron': [6, 4, 0.6666666666666666],
 'Alina': [0, 0, 0],
 'André': [24, 17, 0.7083333333333334],
 'Basti': [0, 0, 0],
 'Chris': [8, 2, 0.25],
 'Cuba': [0, 0, 0],
 'Daisy': [7, 5, 0.7142857142857143],
 'Eggi': [0, 0, 0],
 'Emilio': [34, 21, 0.6176470588235294],
 'Friedrich': [10, 6, 0.6],
 'Gina': [0, 0, 0],
 'Hannes': [0, 0, 0],
 'Henny': [0, 0, 0],
 'Jonte': [0, 0, 0],
 'Konni': [0, 0, 0],
 'Konrad': [0, 0, 0],
 'Leon': [3, 3, 1.0],
 'Lin': [0, 0, 0],
 'Lorenz': [0, 0, 0],
 'Luca': [6, 2, 0.3333333333333333],
 'Marian': [12, 5, 0.4166666666666667],
 'Marie': [0, 0, 0],
 'Max': [0, 0, 0],
 'Merlin': [8, 2, 0.25],
 'Michi': [0, 0, 0],
 'Moritz': [0, 0, 0],
 'Mümmel': [4, 2, 0.5],
 'Nikita': [2, 0, 0.0],
 'Nora': [0, 0, 0],
 'Ole': [0, 0, 0],
 'Olek': [0, 0, 0],
 'Phia': [0, 0, 0],
 'Philipp': [0, 0, 0],
 'Richard': [3, 0, 0.0],
 'Tabea': [0, 0, 0],
 'Timo': [0, 0, 0],
 'Tobi': [0, 0, 0],
 'Valdemar': [21, 10, 0.47619047619047616],
 'Verena': [0, 0, 0],
 'Vincent': [0, 0, 0],
 'Zoë': [

In [131]:
andre_partner, andre_opponents3 = winrate('André')

Total games: 118
Winrate André: 53.39%



In [132]:
andre_opponents3

{'Aaron': [5, 2, 0.4],
 'Alina': [1, 0, 0.0],
 'André': [0, 0, 0],
 'Basti': [0, 0, 0],
 'Chris': [24, 16, 0.6666666666666666],
 'Cuba': [0, 0, 0],
 'Daisy': [20, 12, 0.6],
 'Eggi': [2, 2, 1.0],
 'Emilio': [40, 19, 0.475],
 'Friedrich': [5, 2, 0.4],
 'Gina': [2, 2, 1.0],
 'Hannes': [0, 0, 0],
 'Henny': [0, 0, 0],
 'Jonte': [2, 0, 0.0],
 'Konni': [1, 0, 0.0],
 'Konrad': [0, 0, 0],
 'Leon': [2, 1, 0.5],
 'Lin': [0, 0, 0],
 'Lorenz': [1, 0, 0.0],
 'Luca': [4, 3, 0.75],
 'Marian': [31, 18, 0.5806451612903226],
 'Marie': [3, 3, 1.0],
 'Max': [2, 1, 0.5],
 'Merlin': [11, 8, 0.7272727272727273],
 'Michi': [0, 0, 0],
 'Moritz': [1, 1, 1.0],
 'Mümmel': [6, 2, 0.3333333333333333],
 'Nikita': [2, 2, 1.0],
 'Nora': [4, 3, 0.75],
 'Ole': [3, 1, 0.3333333333333333],
 'Olek': [2, 1, 0.5],
 'Phia': [38, 17, 0.4473684210526316],
 'Philipp': [9, 6, 0.6666666666666666],
 'Richard': [0, 0, 0],
 'Tabea': [0, 0, 0],
 'Timo': [0, 0, 0],
 'Tobi': [1, 1, 1.0],
 'Valdemar': [36, 16, 0.4444444444444444],
 'Veren

In [12]:
def fenster_winrate():
    df = spiele.copy(deep=True)
    df = df[df['fenster'] == 1]
    # since we made sure that team 1 is always fenster, we can just sum up the wins of team1 and divide by the total number of games which have a value in fenster
    print(sum(df['WLt1'])/len(df))
    return

In [14]:
fenster_winrate()

0.5192307692307693


## Test Area

Maybe represent teams as dicts of player names?

In [27]:


dict_test = {player: [0, 0] for player in players}
#dict_test['Team1'] = [2, 1]
#dict_test['Team1'][1] += 1

dict_test['Chris'][1] += 1
dict_test

{'Aaron': [0, 0],
 'Alina': [0, 0],
 'André': [0, 0],
 'Basti': [0, 0],
 'Chris': [0, 1],
 'Cuba': [0, 0],
 'Daisy': [0, 0],
 'Eggi': [0, 0],
 'Emilio': [0, 0],
 'Friedrich': [0, 0],
 'Gina': [0, 0],
 'Hannes': [0, 0],
 'Henny': [0, 0],
 'Jonte': [0, 0],
 'Konni': [0, 0],
 'Konrad': [0, 0],
 'Leon': [0, 0],
 'Lin': [0, 0],
 'Lorenz': [0, 0],
 'Luca': [0, 0],
 'Marian': [0, 0],
 'Marie': [0, 0],
 'Max': [0, 0],
 'Merlin': [0, 0],
 'Michi': [0, 0],
 'Moritz': [0, 0],
 'Mümmel': [0, 0],
 'Nikita': [0, 0],
 'Nora': [0, 0],
 'Ole': [0, 0],
 'Olek': [0, 0],
 'Phia': [0, 0],
 'Philipp': [0, 0],
 'Richard': [0, 0],
 'Tabea': [0, 0],
 'Timo': [0, 0],
 'Tobi': [0, 0],
 'Valdemar': [0, 0],
 'Verena': [0, 0],
 'Vincent': [0, 0],
 'Zoë': [0, 0]}

In [84]:
test1 = ''
if test1:
    print('hallo')
test2 = "Chris"
if test2:
    print('hallo2')
test_dict = {}
if test1 and test1 in test_dict:
    print('sos')
else: print('sus')

hallo2
sus


In [90]:

#partner_test = pd.DataFrame(partner_andre, columns = ['Name', 'Total Games'])
#partner_test['Wins'] = partner_wins_andre
#partner_test.set_index('Name', inplace = True)

#partner_test.loc['Phia', 'Wins'] = partner_wins_andre['Phia']
#for key in partner_wins_andre:
#    partner_test.loc[key, 'Wins'] = partner_wins_andre[key]
#partner_test['Wins'].fillna(0, inplace = True)
#partner_test['Winrate'] = partner_test['Wins']/partner_test['Total Games']
#partner_test

In [89]:
andre, partner_andre = winrate('André') 
partner_andre.sort_values('Winrate', ascending = False)

Total games: 118
Winrate André: 53.39%



,Total Games,Wins,Winrate
Name,,,
Henny,3,8.0,2.666667
Marie,3,6.0,2.000000
Valdemar,16,21.0,1.312500
Luca,9,11.0,1.222222
Philipp,4,4.0,1.000000
Mümmel,5,5.0,1.000000
Marian,15,11.0,0.733333
Phia,24,17.0,0.708333
Emilio,16,10.0,0.625000


In [9]:
andre, partner_andre = winrate('André') 
partner_andre

Total games: 118
Winrate André: 53.39%



[('Phia', 24),
 ('Chris', 20),
 ('Valdemar', 16),
 ('Emilio', 16),
 ('Marian', 15),
 ('Luca', 9),
 ('Daisy', 7),
 ('Merlin', 7),
 ('Aaron', 6),
 ('Mümmel', 5),
 ('Michi', 4),
 ('Philipp', 4),
 ('Basti', 3),
 ('Marie', 3),
 ('Henny', 3),
 ('Timo', 2),
 ('Gina', 2)]

In [41]:
for player in players:
    winrate(player)

Total games: 21
Winrate Aaron: 47.62%

Total games: 3
Winrate Alina: 66.67%

Total games: 118
Winrate André: 53.39%

Total games: 8
Winrate Basti: 25.00%

Total games: 87
Winrate Chris: 47.13%

Total games: 3
Winrate Cuba: 0.00%



KeyError: 'Wins'

In [ ]:
# remove empty string
partner_andre = [x for x in partner_andre if x]
# remove the player themselves
partner_andre = [x for x in partner_andre if x!= 'André']
partner_andre
partners = Counter(partner_andre).most_common()
partners

In [18]:
# OLD WINRATE
def winrate2(player):
    df = spiele.copy(deep = True)
    # use this pattern to get rid of unwanted whitespaces and to make sure 'Ole' is not found when searching for 'Olek'
    pattern = rf'\b{player}\b'
    # important: reset the index because otherwise it will not start at 0 and the for loop wont work
    # drop = True: don't include the old indexes as a column
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    team = -1
    games = 0
    wins = 0
    partner = []
    player_number = 'p0'
    partner_games = {}
    partner_wins = {}
    partner1 = ''
    partner2 = ''
    for i in range(len(df)):
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p2'])
            partner.append(df.loc[i, 'p3'])
            #if df.loc[i, 'p2'] in partner_games
            #partner_games[df.loc[i, 'p2']] += 1
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p1'])
            partner.append(df.loc[i, 'p3'])   
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p1'])
            partner.append(df.loc[i, 'p2'])
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p5'])
            partner.append(df.loc[i, 'p6']) 
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p4'])
            partner.append(df.loc[i, 'p6']) 
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p4'])
            partner.append(df.loc[i, 'p5']) 
        else:
            print("Player {player} not found in row {row}".format(player = player, row = i))
        if team == 1:
            games += 1
            wins = wins + df.loc[i, 'WLt1']
        elif team == 2:
            games += 1
            wins = wins + df.loc[i, 'WLt2']
        else:
            print("Team not set correctly on Player {player} in row {row}".format(player = player, row = i))
    winrate = wins/games*100        
    print('Total games: {games}'.format(games=games))
    print('Winrate {player}: {winrate:.2f}%\n'.format(player=player, winrate=winrate))
    # remove empty string
    partner = [x for x in partner if x]
    # remove the player themselves, maybe not needed anymore
    #partner = [x for x in partner if x!= player]
    # count the occurences of partners and return them sorted by most frequent
    partner_count = Counter(partner).most_common()
    return df, partner_count

In [ ]:
# OLD WINRATE
def winrate3(player):
    df = spiele.copy(deep = True)
    # use this pattern to get rid of unwanted whitespaces and to make sure 'Ole' is not found when searching for 'Olek'
    pattern = rf'\b{player}\b'
    # important: reset the index because otherwise it will not start at 0 and the for loop wont work
    # drop = True: don't include the old indexes as a column
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    team = -1
    games = 0
    wins = 0
    partner = []
    player_number = 'p0'
    partner_games = {}
    partner_wins = {}
    partner1 = ''
    partner2 = ''
    for i in range(len(df)):
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p2']:
                partner1 = df.loc[i, 'p2']
                partner.append(partner1)
                #if partner1 in partner_games:
                #    partner_games[partner1] += 1
                #else: partner_games[partner1] = 1
            if partner2 != df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)  
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p2']:
                partner2 = df.loc[i, 'p2']
                partner.append(partner2)
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p5']:
                partner1 = df.loc[i, 'p5']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p5']:
                partner2 = df.loc[i, 'p5']
                partner.append(partner2)
        else:
            print("Player {player} not found in row {row}".format(player = player, row = i))
        if team == 1:
            games += 1
            current_win = df.loc[i, 'WLt1']
            if current_win:
                wins = wins + 1
                if partner1 in partner_wins:
                    partner_wins[partner1] += 1
                else: partner_wins[partner1] = 1
                if partner2 in partner_games:
                    partner_games[partner2] += 1
                else: partner_games[partner2] = 1
        elif team == 2:
            games += 1
            current_win = df.loc[i, 'WLt2']
            if current_win:
                wins = wins + 1
                if partner1 in partner_wins:
                    partner_wins[partner1] += 1
                else: partner_wins[partner1] = 1
                if partner2 in partner_games:
                    partner_games[partner2] += 1
                else: partner_games[partner2] = 1
        else:
            print("Team not set correctly on Player {player} in row {row}".format(player = player, row = i))
    winrate = wins/games*100        
    print('Total games: {games}'.format(games=games))
    print('Winrate {player}: {winrate:.2f}%\n'.format(player=player, winrate=winrate))
    # remove empty string
    partner = [x for x in partner if x]
    # remove the player themselves, maybe not needed anymore
    #partner = [x for x in partner if x!= player]
    # count the occurences of partners and return them sorted by most frequent
    partner_count = Counter(partner).most_common()
    return df, partner_count, partner_wins

In [ ]:
def winrate4(player):
    df = spiele.copy(deep = True)
    # use this pattern to get rid of unwanted whitespaces and to make sure 'Ole' is not found when searching for 'Olek'
    pattern = rf'\b{player}\b'
    # important: reset the index because otherwise it will not start at 0 and the for loop wont work
    # drop = True: don't include the old indexes as a column
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    team = -1
    games = 0
    wins = 0
    partner = []
    partner = {player: [0, 0] for player in players}
    player_number = 'p0'
#    partner_games = {}
    partner_wins = {}
    partner1 = ''
    partner2 = ''
    for i in range(len(df)):
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            # add both other players of team 1 as partners
            if df.loc[i, 'p2']:
                partner1 = df.loc[i, 'p2']
                partner.append(partner1)
                partner[partner1][0] += 1 
                #if partner1 in partner_games:
                #    partner_games[partner1] += 1
                #else: partner_games[partner1] = 1
            if df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)
                partner[partner2][0] += 1 
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            # add both other players of team 1 as partners
            if df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
                partner[partner1][0] += 1 
            if df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)
                partner[partner2][0] += 1  
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            # add both other players of team 1 as partners
            if df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
                partner[partner1][0] += 1 
            if df.loc[i, 'p2']:
                partner2 = df.loc[i, 'p2']
                partner.append(partner2)
                partner[partner2][0] += 1 
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            # add both other players of team 1 as partners
            if df.loc[i, 'p5']:
                partner1 = df.loc[i, 'p5']
                partner.append(partner1)
                partner[partner1][0] += 1 
            if df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
                partner[partner2][0] += 1 
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            # add both other players of team 1 as partners
            if df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
                partner[partner1][0] += 1 
            if df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
                partner[partner2][0] += 1 
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            # add both other players of team 1 as partners
            if df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
                partner[partner1][0] += 1 
            if df.loc[i, 'p5']:
                partner2 = df.loc[i, 'p5']
                partner.append(partner2)
                partner[partner2][0] += 1 
        else:
            print("Player {player} not found in row {row}".format(player = player, row = i))
        if team == 1:
            games += 1
            current_win = df.loc[i, 'WLt1']
            if current_win:
                wins = wins + 1
                if partner1:
                    partner[partner1][1] += 1 
                    if partner1 in partner_wins:
                        partner_wins[partner1] += 1
                    else: partner_wins[partner1] = 1
                if partner2:
                    partner[partner2][1] += 1 
                    if partner2 in partner_wins:
                        partner_wins[partner2] += 1
                    else: partner_wins[partner2] = 1
        elif team == 2:
            games += 1
            current_win = df.loc[i, 'WLt2']
            if current_win:
                wins = wins + 1
                if partner1:  
                    partner[partner1][1] += 1 
                    if partner1 in partner_wins:
                        partner_wins[partner1] += 1
                    else: partner_wins[partner1] = 1
                if partner2:
                    partner[partner2][1] += 1 
                    if partner2 in partner_wins:
                        partner_wins[partner2] += 1
                    else: partner_wins[partner2] = 1
        else:
            print("Team not set correctly on Player {player} in row {row}".format(player = player, row = i))
    winrate = wins/games*100        
    print('Total games: {games}'.format(games=games))
    print('Winrate {player}: {winrate:.2f}%\n'.format(player=player, winrate=winrate))
    # remove empty string
    partner = [x for x in partner if x]
    # remove the player themselves, maybe not needed anymore
    #partner = [x for x in partner if x!= player]
    # count the occurences of partners and return them sorted by most frequent
    partner_count = Counter(partner).most_common()
    # pack the partner names & number of total games into a df 
    partner_df = pd.DataFrame(partner_count, columns = ['Name', 'Total Games'])
    # make the names the index
    partner_df.set_index('Name', inplace = True)
    # add the parter wins
    for key in partner_wins:
        partner_df.loc[key, 'Wins'] = partner_wins[key]
    partner_df['Wins'].fillna(0, inplace = True)
    partner_df['Winrate'] = partner_df['Wins']/partner_df['Total Games']
    # add winrate to partner
    for elem in partner:
        games_tmp = partner[elem][0]
        wins_tmp = partner[elem][1]
        # handle division by 0
        if games_tmp !=0:
            winrate_tmp = wins_tmp / games_tmp
        else: 
            winrate_tmp = 0
        partner[elem].append(winrate_tmp)
    return partner_df, partner